In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"
import numpy as np
from matplotlib import pyplot as plt
import torch
from sparse_causal_model_learner_rl.sacred_gin_tune.sacred_wrapper import load_config_files
from sparse_causal_model_learner_rl.learners.rl_learner import CausalModelLearnerRL
from sparse_causal_model_learner_rl.config import Config
from tqdm.auto import tqdm
from torch import nn
import gin

In [2]:
gin.enter_interactive_mode()
load_config_files(['../keychest/config/5x5_1f1c1k.gin','../sparse_causal_model_learner_rl/configs/rec_nonlin_gnn.gin'])

['5x5_1f1c1k', 'rec_nonlin_gnn']

In [3]:
learner = CausalModelLearnerRL(Config())

Make environment KeyChest-v0 None {}


In [4]:
learner.create_trainables()

In [5]:
learner.collect_steps()

In [6]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/test_vis_model')

In [7]:
?writer.add_graph

Signature: writer.add_graph(model, input_to_model=None, verbose=False)
Docstring: <no docstring>
File:      ~/miniconda3/envs/causal/lib/python3.7/site-packages/torch/utils/tensorboard/writer.py
Type:      method


In [8]:
o_t = learner._context['obs_x']
f_t = learner.decoder(o_t)
a_t = learner._context['action_x']

In [9]:
writer.add_graph(learner.model, (f_t, a_t))

/home/sergei/git/science/causality-disentanglement/sparse_causal_model_learner_rl/trainable/model.py:93: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert f_t.shape[1] == self.n_features, f"Wrong f_t shape {f_t.shape}"
/home/sergei/git/science/causality-disentanglement/sparse_causal_model_learner_rl/trainable/model.py:94: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert a_t.shape[1] == self.n_actions, f"Wrong a_t shape {a_t.shape}"
/home/sergei/git/science/causality-disentanglement/sparse_causal_model_learner_rl/trainable/model.py:95: TracerWarn

In [10]:
writer.add_graph(learner.decoder, o_t)

In [11]:
writer.flush()

In [12]:
loss_fcn = learner.config.get('losses')['sparsity']['fcn']#(**learner._context)['loss']

In [13]:
loss_fcn

<function sparse_causal_model_learner_rl.loss.losses.sparsity_loss(model, device, add_reg=True, ord=1, eps=1e-08, add_inv=True, **kwargs)>

In [14]:
class LossModule(nn.Module):
    def __init__(self, loss_fcn):
        super(LossModule, self).__init__()
        self.decoder = learner.decoder
        self.model = learner.model
        self.loss_fcn = loss_fcn
        
    def forward(self):
        return self.loss_fcn(self.model, learner.device)['loss']

In [15]:
?writer.add_graph

Signature: writer.add_graph(model, input_to_model=None, verbose=False)
Docstring: <no docstring>
File:      ~/miniconda3/envs/causal/lib/python3.7/site-packages/torch/utils/tensorboard/writer.py
Type:      method


In [16]:
writer.add_graph(LossModule(loss_fcn), [])#, (learner._context['obs_x'].detach(),
                                       # learner._context['obs_y'].detach(),
                                       # learner._context['action_x'].detach()))

/home/sergei/git/science/causality-disentanglement/sparse_causal_model_learner_rl/trainable/model.py:88: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert w.shape[1] == self.n_features + self.n_actions
/home/sergei/git/science/causality-disentanglement/sparse_causal_model_learner_rl/trainable/model.py:75: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert wmax.shape[0] == self.n_features + self.n_actions
/home/sergei/miniconda3/envs/causal/lib/python3.7/site-packages/torch/tensor.py:461: RuntimeWarning: Iterating over a tensor might cause the trac

In [17]:
writer.flush()